This notebook performs the experiment, where a DNN is the base model to mitigate. We consider two fairness definitions: statistical parity and equal opportunity. All three compared methods are assessed on statistical parity, while only FairWhere and PROMIS are assessed on equal opportunity. For FairWhere we define the distance for each partition as the absoluted difference of the partition recall and the overall model recall and for statistical parity we use the positive ratio as metric
The three types of audit regions are tested and with the 2 fairness notions result in 6 experiments.

Initially the base DNN is trained. FairWhere continues training on the training set with the goal of mitigating spatial bias. At the start of fairness training, each sampled partitioning is used for 5 epochs before transitioning to the next, iterating over 120 different samples (equivalent to five full enumerations over all 24 partitioning candidates). Subsequently, each epoch samples a new partitioning, continuing for 720 samples (equivalent to thirty full enumerations). In total, each partitioning is expected to be used for 50 epochs throughout
the training process.

The predictions of the base and corrected (by FairWhere) model are saved for later analysis.

The maximum budget, given as input for PROMIS and SpatialFlips is computed by the absolute difference of the predictions on the validation set between the base DNN and FairWhere models. 

PROMIS methods apply the thresholds adjustment approach using the validation set, while SpatialFlips apply flips directly on the test set.

The "pretrained" PROMIS, SpatialFlip models are saved for later analysis on the test set.

You may opt which experiment to run by uncommenting the relate part in the section choose experiment to run. To rerun an experiment restart the notebook (for reprodicaability)

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score,recall_score, precision_score,f1_score, balanced_accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
import tensorflow as tf

import numpy as np
import random
import time
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

from tqdm.notebook import tqdm
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..')))

from methods.models.optimization_model import SpatialOptimFairnessModel 
from methods.models.spatial_flip_model import SpatialFlipFairnessModel
from utils.results_names_utils import combine_world_info
from utils.data_utils import get_y, read_scanned_regs, get_pos_info_regions
import ast

## Choose Experiment to Run

### Equal Opportunity

In [ ]:
# fair_notion = "equal_opportunity" 
# partioning_type_name = "5_x_5"
# n_flips_start = 400
# overlap=True

# fair_notion = "equal_opportunity" 
# partioning_type_name = "non_overlap_k_8"
# n_flips_start = 400
# overlap=False

# fair_notion = "equal_opportunity" 
# partioning_type_name = "overlap_k_10_radii_4"
# n_flips_start = 1000
# overlap=True

### Statistical Parity

In [2]:
# fair_notion = "statistical_parity" 
# partioning_type_name = "5_x_5"
# n_flips_start = 1200
# overlap=True

# fair_notion = "statistical_parity" 
# partioning_type_name = "non_overlap_k_8"
# n_flips_start = 900
# overlap=False

# fair_notion = "statistical_parity" 
# partioning_type_name = "overlap_k_10_radii_4"
# n_flips_start = 3000
# overlap=True

In [3]:
data_path = "../../data/"
prediction_dir = f'{data_path}predictions/' # folder in data path to save the base model predictions
partitioning_dir = f'{data_path}partitionings/' # folder in data path to save the partitionings
preprocess_dir = f'{data_path}preprocess/'
dataset_name = "crime"
clf_name = "dnn"
results_path = "../../results/dnn_exp/"


In [4]:
res_desc_label, partioning_name, prediction_name = combine_world_info(
    dataset_name, partioning_type_name, clf_name
)
results_path = f"{results_path}{res_desc_label}/"
keras_models_dir = f'{results_path}keras_models/{fair_notion}/' # folder to save base and where model
sp_flip_models_dir = f'{results_path}spatial_flip_models/{fair_notion}/'
sp_opt_models_dir = f'{results_path}spatial_optim_models/{fair_notion}/'
base_model_fname = f"{data_path}clf/dnn_{dataset_name}.keras"
for dir in [results_path, keras_models_dir]:
    os.makedirs(dir, exist_ok=True)

In [5]:
SCORE_LABEL = 'recall' if fair_notion == 'equal_opportunity' else 'pr'
print(f"SCORE_LABEL: {SCORE_LABEL}")

SCORE_LABEL: pr


In [6]:
BATCH_SIZE = 64*64
LEARNING_RATE = 0.0001
LEARNING_RATE_INIT = LEARNING_RATE
EPOCH_TRAIN = 100

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [7]:
#Get F1 list of all partitions
def get_weighted_f1_one(model, X_Test, y_Test, all_partitioning_data_list, index1, index2):
    w_f1_list = np.zeros((index1 * index2), dtype='float')

    y_pred = np.argmax(model.predict(X_Test, batch_size=BATCH_SIZE),axis = -1)

    data_list = all_partitioning_data_list[(index1, index2)]

    for i in range(index1 * index2):
        if data_list[i].size == 0:
            continue
        y_pred_partition = y_pred[data_list[i]]
        y_test_partition = y_Test[data_list[i]]

        f1_value = f1_score(y_test_partition,y_pred_partition,zero_division = 0)
        w_f1_list[i] = f1_value

    return w_f1_list

#Get recall list of all partitions
def get_weighted_recall(model, X_Test, y_Test, all_partitioning_data_list, index1, index2):
    w_tpr_list = np.zeros((index1 * index2), dtype='float')

    y_pred = np.argmax(model.predict(X_Test, batch_size=BATCH_SIZE),axis = -1)

    data_list = all_partitioning_data_list[(index1, index2)]

    for i in range(index1 * index2):
        if data_list[i].size == 0:
            continue
        y_pred_partition = y_pred[data_list[i]]
        y_test_partition = y_Test.iloc[data_list[i]]

        rec = recall_score(y_test_partition,y_pred_partition,zero_division = 0)
        w_tpr_list[i] = rec

    return w_tpr_list

def get_pr(y_true, y_pred):
    return np.sum(y_pred) / len(y_pred)


def get_weighted_pr(
    model, X_Test, y_Test, all_partitioning_data_list, index1, index2
):
    w_pr_list = np.zeros((index1 * index2), dtype="float")

    y_pred = np.argmax(model.predict(X_Test, batch_size=BATCH_SIZE), axis=-1)

    data_list = all_partitioning_data_list[(index1, index2)]

    for i in range(index1 * index2):
        if data_list[i].size == 0:
            continue
        y_pred_partition = y_pred[data_list[i]]

        pr = get_pr(None, y_pred_partition)
        w_pr_list[i] = pr

    return w_pr_list

score_labels_to_funcs = {
    "pr": {
        "score_func": get_pr,
        "weighted_score_func": get_weighted_pr
    },
    "recall": {
        "score_func": recall_score,
        "weighted_score_func": get_weighted_recall
    }
}
score_func = score_labels_to_funcs[SCORE_LABEL]["score_func"]
weighted_score_func = score_labels_to_funcs[SCORE_LABEL]["weighted_score_func"]

def dice(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersect = tf.reduce_sum(y_pred * y_true, axis=0) + K.epsilon()
    denominator = tf.reduce_sum(y_pred, axis=0) + tf.reduce_sum(y_true, axis=0)
    dice_scores = 2 * intersect / (denominator + tf.keras.backend.epsilon())
    return 1 - dice_scores

def custom_loss(y_true, y_pred):
    loss = dice(y_true, y_pred)
    return loss

## Read Data

In [8]:
# Load data
X_train = pd.read_csv(f"{preprocess_dir}X_train_crime.csv")
X_val = pd.read_csv(f"{preprocess_dir}X_val_crime.csv")
X_test = pd.read_csv(f"{preprocess_dir}X_test_crime.csv")
y_train = pd.read_csv(f"{preprocess_dir}y_train_crime.csv")
y_val = pd.read_csv(f"{preprocess_dir}y_val_crime.csv")
y_test = pd.read_csv(f"{preprocess_dir}y_test_crime.csv")

In [ ]:
train_partitioning_id_df = pd.read_csv(
    f"{partitioning_dir}train_{partioning_name}_partitioning_ids.csv"
)

train_partitioning_id_df["id"] = train_partitioning_id_df["id"].apply(
    ast.literal_eval
)
train_partitioning_id_df["partitioning"] = train_partitioning_id_df[
    "partitioning"
].apply(ast.literal_eval)

# Get the partitioning data

train_ids = train_partitioning_id_df["id"].tolist()
X_train_parts = train_partitioning_id_df["partitioning"].tolist()

X_train_parts_np = []
for i in range(len(X_train_parts)):
    cur_partitioning = []
    for partition in X_train_parts[i]:
        cur_partitioning.append(np.array(partition))
    X_train_parts_np.append(cur_partitioning)

X_train_id = {id: part for id, part in zip(train_ids, X_train_parts_np)}

PARTITIONINGS = train_ids
print("Partitioning: ", PARTITIONINGS)

##  Train Base Model

In [12]:
initializer = tf.keras.initializers.TruncatedNormal(stddev=0.5, seed=SEED)
base_model = Sequential([
    Dense(100, activation='elu', input_dim=X_train.shape[1], kernel_initializer=initializer),
    BatchNormalization(),
    Dropout(0.1),
    Dense(50, activation='elu', kernel_initializer=initializer),
    BatchNormalization(),
    Dropout(0.1),
    Dense(2, activation='softmax', kernel_initializer=initializer),
])

print(base_model.summary())


optimizer = Adam(learning_rate=LEARNING_RATE)
global optimizer
base_model.compile(optimizer=optimizer, loss=custom_loss, metrics=['accuracy'])

# Start Training
history = base_model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCH_TRAIN,
    shuffle=True,
    validation_data=(X_val, y_val),
)

# We do not save the model since the same reproducible 
# trained model has been save in create_worlds.py

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.show()

In [12]:
base_model.save(base_model_fname)

In [ ]:
base_model = load_model(
    base_model_fname,
    custom_objects={"custom_loss": custom_loss} 
)

In [13]:
#Predict the label
y_train_pred_base = base_model.predict(X_train, batch_size=BATCH_SIZE)
y_val_pred_base =  base_model.predict(X_val, batch_size=BATCH_SIZE)
y_test_pred_base =  base_model.predict(X_test, batch_size=BATCH_SIZE)

#Change Softmax to class
class_y_train_pred_base = np.argmax(y_train_pred_base, axis=-1)
class_y_val_pred_base = np.argmax(y_val_pred_base, axis=-1)
class_y_test_pred_base = np.argmax(y_test_pred_base, axis=-1)

#Prediction Value Count
print(np.unique(class_y_train_pred_base,return_counts=True))
print(np.unique(class_y_val_pred_base,return_counts=True))
print(np.unique(class_y_test_pred_base,return_counts=True))

base_test_acc = accuracy_score(y_test,class_y_test_pred_base)
base_test_bal_acc = balanced_accuracy_score(y_test,class_y_test_pred_base)
base_test_rec = recall_score(y_test,class_y_test_pred_base)
base_test_pre = precision_score(y_test,class_y_test_pred_base)
base_test_f1 = f1_score(y_test,class_y_test_pred_base)


print(f"Acc train: {accuracy_score(y_train,class_y_train_pred_base)}, Acc val: {accuracy_score(y_val,class_y_val_pred_base)}, Acc test: {accuracy_score(y_test,class_y_test_pred_base)}")
print(f"Rec train: {recall_score(y_train,class_y_train_pred_base)}, Rec val: {recall_score(y_val,class_y_val_pred_base)} Rec test: {recall_score(y_test,class_y_test_pred_base)}")
print(f"Pre train: {precision_score(y_train,class_y_train_pred_base)}, Pre val: {precision_score(y_val,class_y_val_pred_base)}, Pre test: {precision_score(y_test,class_y_test_pred_base)}")
print(f"F1 train: {f1_score(y_train,class_y_train_pred_base)}, F1 val: {f1_score(y_val,class_y_val_pred_base)}, F1 test: , {f1_score(y_test,class_y_test_pred_base)}")

## Save Base Model Predictions and Generated Probabilities

In [ ]:
pos_class_prob_idx = 1  # Positive class index

#Predict the label
y_train_pred_base = base_model.predict(X_train, batch_size=BATCH_SIZE)

#Change Softmax to class
class_y_train_pred_base = np.argmax(y_train_pred_base, axis=-1)
GLOBAL_MEAN_base = score_func(y_train, class_y_train_pred_base)

y_train_base_pred_proba = base_model.predict(X_train, batch_size=BATCH_SIZE)[:, pos_class_prob_idx]
y_val_base_pred_proba = base_model.predict(X_val, batch_size=BATCH_SIZE)[:, pos_class_prob_idx]
y_test_base_pred_proba = base_model.predict(X_test, batch_size=BATCH_SIZE)[:, pos_class_prob_idx]

preds_df = pd.DataFrame({
    "pred": class_y_train_pred_base,
    "pred_proba": y_train_base_pred_proba,
})

val_preds_df = pd.DataFrame({
    "pred": class_y_val_pred_base,
    "prob": y_val_base_pred_proba
})

test_preds_df = pd.DataFrame({
    "pred": class_y_test_pred_base,
    "prob": y_test_base_pred_proba
})

preds_df.to_csv(f"{prediction_dir}train_{prediction_name}.csv", index=False)
val_preds_df.to_csv(f"{prediction_dir}val_{prediction_name}.csv", index=False)
test_preds_df.to_csv(f"{prediction_dir}test_{prediction_name}.csv", index=False)

105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


## Motigate Spatial Bias with FairWhere Model
Technique used in the Fairness by Where Paper, Bi-Level Framework, train on different random partitions

In [17]:
#Get Data for one way of partitioning
def get_partition_data(index1, index2, X_data, y_data, all_partitioning_data_list):
    X_train_list = []
    y_train_list = []

    data_list = all_partitioning_data_list[(index1, index2)]
    #print(data_list)
    for i in range(index1 * index2):
        if data_list[i].size == 0:
            continue
        X_train_list.append(X_data.iloc[data_list[i]])
        y_train_list.append(y_data.iloc[data_list[i]])

    #print("get index {} partition data finished".format((index1,index2)))

    return X_train_list, y_train_list


### Self-adpated learning rate, defined in the fairness by where paper
def set_lr_weight(index1, index2, w_scores):
    global lr_list, GLOBAL_MEAN_base

    lr_list = np.zeros((index1 * index2))

    #print("w_scores: ", w_scores)

    lr_list = (GLOBAL_MEAN_base - w_scores)
    # percentage
    lambda_value = LEARNING_RATE_INIT #0.0005

    lr_list = tf.nn.relu(lr_list)
    lr_list = lr_list.numpy()

    if np.sum(lr_list) > 0:
        lr_list = lr_list / np.sum(lr_list) * lambda_value * (index1 * index2)

        # maximum to lambda_value
        lr_list = lr_list / (np.amax(lr_list) / lambda_value)

    # avoid zero
    lr_list += K.epsilon()
    #print("Final lr_list:", lr_list)
    return lr_list

In [14]:
#Use intial epoch in case we need to train on different partitions
def model_train(model, X_in, y_in, init_epoch_number=0, epoch_train=EPOCH_TRAIN, X_val=None, y_val=None, verbose=0):  #
    '''
    Input model is complied!
    '''

    if X_val is None or y_val is None:
        return model.fit(
            X_in, 
            y_in, 
            batch_size=BATCH_SIZE,
            initial_epoch=init_epoch_number, 
            epochs=init_epoch_number + epoch_train, 
            shuffle=True,
            verbose=verbose,
        )
    else:
        return model.fit(
            X_in, 
            y_in, 
            batch_size=BATCH_SIZE,
            initial_epoch=init_epoch_number, 
            epochs=init_epoch_number + epoch_train, 
            shuffle=True,
            validation_data=(X_val, y_val), 
            verbose=verbose,
        )
    
# Bi-level
base_model = load_model(
    base_model_fname,
    custom_objects={"custom_loss": custom_loss}
)
start_time = time.time()

loop_list = [4, 30]
epoch_list = [5, 1]

for i in tqdm(range(len(loop_list)), desc="Outer Loop"):

    epochs = epoch_list[i]
    loops = loop_list[i]

    for l in tqdm(range(loops), desc="Training Loops", leave=False):

        random.shuffle(PARTITIONINGS)

        for j in tqdm(range(len(PARTITIONINGS)), desc="Partitions", leave=False):
            (index1, index2) = PARTITIONINGS[j]

            X_train_part, y_train_part = get_partition_data(index1, index2, X_train, y_train, X_train_id)

            w_scores = weighted_score_func(base_model, X_train, y_train, X_train_id, index1, index2)
            lr_list = set_lr_weight(index1, index2, w_scores,)

            for e in tqdm(range(epochs), desc="Epochs", leave=False):

                for p in range(len(X_train_part)):

                    LEARNING_RATE = lr_list[p]
                    for var in optimizer.variables:
                        var.assign(tf.zeros_like(var))

                    base_model(tf.ones((1, X_train.shape[1])))
                    history = model_train(
                        base_model, 
                        X_train_part[p], 
                        y_train_part[p], 
                        init_epoch_number=e, 
                        epoch_train=1, 
                        X_val=X_val, 
                        y_val=y_val, 
                        verbose=0
                    )

where_fit_time = time.time() - start_time
print(f"Time: {where_fit_time:.3f} s")

## Save Fit Time and Predictions of Mitigated Model

In [19]:
with open(f"{results_path}{dataset_name}_{fair_notion}_where_fit_time.txt", "w") as file:
    file.write(str(where_fit_time))
base_model.save(f"{keras_models_dir}crime_partition_model.keras")

In [10]:
fair_where_model = load_model(f"{keras_models_dir}crime_partition_model.keras", custom_objects={"custom_loss": custom_loss})

In [11]:
y_train_pred_partition = fair_where_model.predict(X_train, batch_size=BATCH_SIZE)
y_val_pred_partition = fair_where_model.predict(X_val, batch_size=BATCH_SIZE)
y_test_pred_partition = fair_where_model.predict(X_test, batch_size=BATCH_SIZE)
class_y_train_pred_partition = np.argmax(y_train_pred_partition, axis=-1)
class_y_val_pred_partition = np.argmax(y_val_pred_partition, axis=-1)
class_y_test_pred_partition = np.argmax(y_test_pred_partition, axis=-1)


pos_class_prob_idx = 1  # Positive class index
y_train_partition_pred_proba = fair_where_model.predict(
    X_train, batch_size=BATCH_SIZE
)[:, pos_class_prob_idx]

y_val_partition_pred_proba = fair_where_model.predict(
    X_val, batch_size=BATCH_SIZE
)[:, pos_class_prob_idx]

y_test_partition_pred_proba = fair_where_model.predict(
    X_test, batch_size=BATCH_SIZE
)[:, pos_class_prob_idx]

train_preds_partition_df = pd.DataFrame(
    {"pred": class_y_train_pred_partition, "prob": y_train_partition_pred_proba}
)

val_preds_partition_df = pd.DataFrame(
    {"pred": class_y_val_pred_partition, "prob": y_val_partition_pred_proba}
)

test_preds_partition_df = pd.DataFrame(
    {"pred": class_y_test_pred_partition, "prob": y_test_partition_pred_proba}
)

train_preds_partition_df.to_csv(
    f"{results_path}{dataset_name}_{fair_notion}_where_model_train_pred.csv",
    index=False,
)
val_preds_partition_df.to_csv(
    f"{results_path}{dataset_name}_{fair_notion}_where_model_val_pred.csv",
    index=False,
)
test_preds_partition_df.to_csv(
    f"{results_path}{dataset_name}_{fair_notion}_where_model_test_pred.csv",
    index=False,
)

105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


## Apply Optimization Correction Methods

### Read Data

In [ ]:
# read data for other spatial bias mitigation models
val_pred_base_df = pd.read_csv(f"{prediction_dir}val_{prediction_name}.csv")
val_pred_partition_df = pd.read_csv(
    f"{results_path}{dataset_name}_{fair_notion}_where_model_val_pred.csv"
)
y_val_df = pd.read_csv(f"{preprocess_dir}y_val_{dataset_name}.csv")
y_val = get_y(y_val_df, "label")
pts_per_region_val_df = read_scanned_regs(
    f"{partitioning_dir}val_{partioning_name}.csv"
)
pts_per_region_val = pts_per_region_val_df["points"].tolist()
y_val_pred_base = get_y(val_pred_base_df, "pred")
y_val_base_pred_proba = get_y(val_pred_base_df, "prob")
y_val_pred_partition = get_y(val_pred_partition_df, "pred")


### Compute the Maximum Budget

In [ ]:
if fair_notion == "equal_opportunity":
    # Get positive class information to consider equal opportunity
    pos_val_y_true_indices, _ = get_pos_info_regions(y_val, pts_per_region_val)
    pos_val_y_pred_base = y_val_pred_base[pos_val_y_true_indices]
    pos_y_val_pred_partition = y_val_pred_partition[pos_val_y_true_indices]

# Calculate the differences between the base and partition model
# to use it as budget for the mitigation models
base_partition_diffs = (
    np.sum(np.abs(y_val_pred_base - y_val_pred_partition))
    if fair_notion == "statistical_parity"
    else np.sum((np.abs(pos_val_y_pred_base - pos_y_val_pred_partition)))
)
print("\nDifferences between base and partition in validation set:")
print("Number of different elements:", base_partition_diffs)
n_flips = base_partition_diffs

### Apply the PROMIS methods

In [ ]:
wlimit=300#3600
max_pr_shift=0.1
promis_methods = [
    "promis_app",
    "promis_opt",
]

for method in promis_methods:
    fair_model = SpatialOptimFairnessModel(method)
    fair_model.multi_fit(
        points_per_region=pts_per_region_val,
        n_flips_start = n_flips_start,
        step=n_flips_start,
        n_flips=n_flips,
        y_pred=y_val_pred_base,
        y_true=y_val if fair_notion == "equal_opportunity" else None,
        y_pred_probs=y_val_base_pred_proba,
        wlimit=wlimit,
        fair_notion=fair_notion,
        overlap=overlap,
        no_of_threads=0,
        verbose=1,
        max_pr_shift=max_pr_shift
    )

    if method == "promis_opt":
        model_save_file = f"{sp_opt_models_dir}/{method}_wlimit_{wlimit}.pkl"
    else:
        model_save_file = f"{sp_opt_models_dir}/{method}.pkl"

    fair_model.save_model(model_save_file)

### Apply Spatial Flip Method (Only for Statistical Parity)

In [18]:
# We apply Spatial Flip Model directly on the test set

if fair_notion == "statistical_parity":
    # read data for other spatial bias mitigation models
    test_pred_base_df = pd.read_csv(f"{prediction_dir}test_{prediction_name}.csv")
    pts_per_region_test_df = read_scanned_regs(
        f"{partitioning_dir}test_{partioning_name}.csv"
    )
    pts_per_region_test = pts_per_region_test_df["points"].tolist()
    y_test_pred_base = get_y(test_pred_base_df, "pred")


    fair_model = SpatialFlipFairnessModel("iter")
    fair_model.multi_fit(
        points_per_region=pts_per_region_test,
        n_flips_start=n_flips_start,
        step=n_flips_start,
        n_flips=n_flips,
        y_pred=y_test_pred_base,
        overlap=overlap,
        verbose=1,
    )

    fair_model.save_model(
        f"{sp_flip_models_dir}/iter.pkl"
    )